# Import libraries

In [1]:
!pip install tfx

     |████████████████████████████████| 2.3MB 8.1MB/s 
     |████████████████████████████████| 1.7MB 17.6MB/s 
     |████████████████████████████████| 2.9MB 40.3MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 
     |████████████████████████████████| 153kB 31.9MB/s 
     |████████████████████████████████| 112kB 35.6MB/s 
     |████████████████████████████████| 1.2MB 29.4MB/s 
     |████████████████████████████████| 1.5MB 21.5MB/s 
     |████████████████████████████████| 2.3MB 36.5MB/s 
     |████████████████████████████████| 17.7MB 194kB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 1.4MB 34.7MB/s 
     |████████████████████████████████| 92kB 7.9MB/s 
     |████████████████████████████████| 9.6MB 27.9MB/s 
     |████████████████████████████████| 409kB 39.9MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 788kB 35.0MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
   

In [2]:
import os
import pprint
import numpy as np
import tempfile
import urllib

import absl
import pandas as pd
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen, ImportExampleGen
from typing import Dict, List, Text
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input
import matplotlib.pyplot as plt
import os  # used for directory operations
import tensorflow as tf
from PIL import Image  # used to read images from directory

import cv2

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Checking Versions and defining variables

In [4]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.4.1
TFX version: 0.29.0


In [5]:
!rm -rf data.*
!rm -rf *trainer.py
!sudo rm -r /content/tfx

rm: cannot remove '/content/tfx': No such file or directory


In [6]:
! cd /content/
! mkdir /content/tfx/
! mkdir /content/tfx/pipelines
! mkdir /content/tfx/metadata
! mkdir /content/tfx/logs
! mkdir /content/tfx/data
! mkdir /content/tfx/serving_model

In [7]:
##Define all constant
_tfx_root = os.path.join(os.getcwd(), 'tfx');        # Create location ~/tfx
_pipeline_root = os.path.join(_tfx_root, 'pipelines');      # Join ~/tfx/pipelines/
_metadata_db_root = os.path.join(_tfx_root, 'metadata.db');    # Join ~/tfx/metadata.db
_log_root = os.path.join(_tfx_root, 'logs');
_model_root = os.path.join(_tfx_root, 'model');
_data_root = os.path.join(_tfx_root, 'data');
_serving_model_dir = os.path.join(_tfx_root, 'serving_model')
_data_filepath = os.path.join(_data_root, "data_trans.csv")

_input_fn_module_file = 'inputfn_trainer.py'
_constants_module_file = 'constants_trainer.py'
_model_trainer_module_file = 'model_trainer.py'

In [8]:
context = InteractiveContext(pipeline_root=_tfx_root)

# Loading training data

In [9]:
# my file path, just like the picture above
cwd = "drive/My Drive/CMP-258 Project/dataset/SampledDatasetFrames/"
# the tfrecord file path, you need to create the folder yourself
recordPath = ""
# the best number of images stored in each tfrecord file
bestNum = 1000
# the index of images flowing into each tfrecord file
num = 0
# the index of the tfrecord file
recordFileNum = 0
# the number of classes of images
keys = ['NoFight', 'Fight']
values = [0, 1]
classes = dict(zip(keys, values))
# name format of the tfrecord files
recordFileName = ("tfx/data/train.tfrecords")
# tfrecord file writer
writer = tf.io.TFRecordWriter(recordFileName)

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def read_image_as_byte_string(path):
  # "image/raw" has bytes value "None" which cannot be decoded as a UTF-8 string.
  image_string = open(str(path), 'rb').read()
  return image_string

print("Creating the 000 tfrecord file")
for name, label in classes.items():
    print(name)
    print(label)
    class_path = os.path.join(cwd, name)
    for img_name in os.listdir(class_path):
        num += 1
        if num > bestNum:
            num = 1
            recordFileNum += 1
            writer = tf.io.TFRecordWriter(recordPath + recordFileNum)
            print("Creating the %.3d tfrecord file" % recordFileNum)
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path, "r")
        img_raw = img.tobytes()
        image_string = read_image_as_byte_string(img_path)
        example = tf.train.Example(features=tf.train.Features(feature={
          "img_raw": _bytes_feature(image_string),
          # "img_raw": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
          "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))}))
        writer.write(example.SerializeToString())
writer.close()

Creating the 000 tfrecord file
NoFight
0
Fight
1


# ExampleGen

In [10]:
tf_record_gen = ImportExampleGen(input=external_input(_data_root))
context.run(tf_record_gen)

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.


ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/content/tfx/ImportExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:11233428,xor_checksum:1620861592,sum_checksum:1620861592"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
artifact = tf_record_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /content/tfx/ImportExampleGen/examples/1


In [12]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(tf_record_gen.outputs['examples'].get()[0].uri, 'Split-train')

# train_file = os.path.join(tf_record_gen.outputs['examples'].get()[0].uri, 'Split-train')
# # Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "img_raw"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\002\001\001\001\001\001\002\001\001\001\002\002\002\002\002\004\003\002\002\002\002\005\004\004\003\004\006\005\006\006\006\005\006\006\006\007\t\010\006\007\t\007\006\006\010\013\010\t\n\n\n\n\n\006\010\013\014\013\n\014\t\n\n\n\377\333\000C\001\002\002\002\002\002\002\005\003\003\005\n\007\006\007\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\377\300\000\021\010\000\360\001@\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDE

# Statistics Gen

In [13]:
statistics_gen = StatisticsGen(
    examples=tf_record_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/content/tfx/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [14]:
context.show(statistics_gen.outputs['statistics'])

# Schema Gen

In [15]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/content/tfx/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [16]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'img_raw',BYTES,required,single,-
'label',INT,required,single,-


# Example validator

In [17]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/content/tfx/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [18]:
context.show(example_validator.outputs['anomalies'])

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [19]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(tf_record_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 1 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "img_raw"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\002\001\001\001\001\001\002\001\001\001\002\002\002\002\002\004\003\002\002\002\002\005\004\004\003\004\006\005\006\006\006\005\006\006\006\007\t\010\006\007\t\007\006\006\010\013\010\t\n\n\n\n\n\006\010\013\014\013\n\014\t\n\n\n\377\333\000C\001\002\002\002\002\002\002\005\003\003\005\n\007\006\007\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\377\300\000\021\010\000\360\001@\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDE

# Transform And Trainer

In [20]:
%%writefile {_input_fn_module_file}

import os
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 20
IMAGE_SIZE = [299, 299]

def make_input_fn(dir_uri, mode, vnum_epochs = None, batch_size = 512):

    def decode_image(image):
      image = tf.image.decode_jpeg(tf.reshape(image, []))
      image = tf.image.resize(image, [299, 299])
      return image

    def read_tfrecord(example):
      tfrecord_format = (
          {
              "img_raw": tf.io.FixedLenFeature([], tf.string),
              "label": tf.io.FixedLenFeature([], tf.int64),
          }
      )
      example = tf.io.parse_example(example[tf.newaxis], tfrecord_format)
      image = decode_image(example["img_raw"])
      label = tf.cast(example["label"], tf.int32)
      label = tf.one_hot(label, 2)
      label = tf.reshape(label, [2,])
      return image, label

    def load_dataset(filenames):
      ignore_order = tf.data.Options()
      ignore_order.experimental_deterministic = False
      tfrecord_filenames = [os.path.join(dir_uri, name)
                            for name in os.listdir(dir_uri)]

      # Create a `TFRecordDataset` to read these files
      dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

      dataset = dataset.with_options(
          ignore_order
      )  # uses data as soon as it streams in, rather than in its original order
      
      dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
      # dataset = dataset.shuffle(2048)
      # dataset = dataset.prefetch(4)
      # dataset = dataset.repeat(2)
      dataset = dataset.batch(BATCH_SIZE)
      return dataset

    def get_dataset(filenames, labeled=True):
      dataset = load_dataset(filenames)
      # print(dataset.shape)
      return dataset

    def _input_fn(v_test=False):
      train_dataset = get_dataset(dir_uri)
      return train_dataset 


    return _input_fn

Writing inputfn_trainer.py


In [21]:
import inputfn_trainer as ift
train_uri = os.path.join(tf_record_gen.outputs['examples'].get()[0].uri, 'Split-train')
train_dataset = ift.make_input_fn(dir_uri = train_uri,
                      mode = tf.estimator.ModeKeys.TRAIN,
                      batch_size = 1)

In [22]:
data = train_dataset()

In [23]:
%%writefile {_model_trainer_module_file}

import tensorflow as tf
import tensorflow.keras as keras
import inputfn_trainer as ift
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
#from data import DataSet
import os.path
# import constants_trainer as ct

from tfx.components.trainer.fn_args_utils import FnArgs
print(tf.__version__)

device = "gpu"
IMAGE_SIZE = [299, 299]

if device == "tpu":
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  # This is the TPU initialization code that has to be at the beginning.
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
else:
  strategy = tf.distribute.MultiWorkerMirroredStrategy()

#Create model
params_default = {
    'lr' : 0.001,
    'beta_1' : 0.99,
    'beta_2' : 0.999,
    'epsilon' : 1e-08,
    'decay' : 0.01,
    'hidden_layers' : 1
}

initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "melanoma_model.h5", save_best_only=True
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

# Define feature columns(Including feature engineered ones )
# These are the features which come from the TF Data pipeline

def create_keras_model():
    # create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=None,
    input_shape=None, classifier_activation="softmax")

    # add a global spatial average pooling layer
    x = base_model.output
    # print("model innput",base_model.input)
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer
    predictions = Dense(2, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in model.layers[:172]:
        layer.trainable = False
    for layer in model.layers[172:]:
        layer.trainable = True

    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    model.compile(
        optimizer=SGD(lr=0.0001, momentum=0.9),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_k_categorical_accuracy'])

    #Print Summary
    # print(model.summary())
    return model

def keras_train_and_evaluate(model, train_dataset, validation_dataset, epochs=100):
  print(train_dataset)
  out = model.fit(
      train_dataset,
      epochs=2,
      validation_data=validation_dataset,
      callbacks=[checkpoint_cb, early_stopping_cb],
  )

  return model

def save_model(model, model_save_path):
  @tf.function
  def serving(img_raw, label):

      payload = {
          'img_raw': img_raw,
          'label': label
      }
      
      ## Predict
      ##IF THERE IS AN ERROR IN NUMBER OF PARAMS PASSED HERE OR DATA TYPE THEN IT GIVES ERROR, "COULDN'T COMPUTE OUTPUT TENSOR"
      predictions = model(payload)
      return predictions

  # serving = serving.get_concrete_function(img_raw=tf.TensorSpec([None,], dtype= tf.int64, name='img_raw'), 
  #                                         label=tf.TensorSpec([None,], dtype= tf.int64, name='label')
  #                                         )

  # version = "1"  #{'serving_default': call_output}
  tf.saved_model.save(
      model,
      model_save_path + "/"
  )

##Main function called by TFX
def run_fn(fn_args: FnArgs):
  print("Inside Trainer")


  train_dataset = ift.make_input_fn(dir_uri = '/content/tfx/ImportExampleGen/examples/1/Split-train',
                      mode = tf.estimator.ModeKeys.TRAIN,
                      batch_size = 128)()

  # # print(train_dataset)

  validation_dataset = ift.make_input_fn(dir_uri = '/content/tfx/ImportExampleGen/examples/1/Split-eval',
                      mode = tf.estimator.ModeKeys.EVAL,
                      batch_size = 512)()

  #Create model
  m_ = create_keras_model()
  tf.keras.utils.plot_model(m_, show_shapes=True, rankdir="LR")

  # print(train_dataset)

  #Train model
  m_ = keras_train_and_evaluate(m_, train_dataset, validation_dataset, 1)

  #Save model with custom signature
  save_model(m_, fn_args.serving_model_dir)


Writing model_trainer.py


In [26]:
trainer = Trainer(
    module_file=os.path.abspath(_model_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=tf_record_gen.outputs['examples'],
    train_args=trainer_pb2.TrainArgs(),
    eval_args=trainer_pb2.EvalArgs(),
    custom_config=({"epochs": 1})
    )

context.run(trainer)

2.4.1
INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
Inside Trainer
87916544/87910968 [==============================] - 1s 0us/step
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.388654 to fit

<BatchDataset shapes: ((None, 299, 299, None), (None, 2)), types: (tf.float32, tf.float32)>
Epoch 1/2
14/14 [==============================] - 148s 10s/step - loss: 0.7875 - accuracy: 0.2179 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.0423 - val_accuracy: 0.4662 - val_top_k_categorical_accuracy: 1.0000
Epoch 2/2
14/14 [==============================] - 135s 10s/step - loss: 0.6197 - accuracy: 0.7577 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.8662 - val_accuracy: 0.4962 - val_top_k_categorical_accuracy: 1.0000
INFO:tensorflow:Assets written to: /content/tfx/Trainer/model/5/Format-Serving/assets

ExecutionResult(
    component_id: Trainer
    execution_id: 5
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "/content/tfx/Trainer/model/5"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/content/tfx/Trainer/model_run/5"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

# Pusher

In [27]:
pusher = Pusher(
    model=trainer.outputs['model'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 6
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 7
        type_id: 16
        uri: "/content/tfx/Pusher/pushed_model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "/content/tfx/serving_model/1620861979"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1620861979"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [28]:
pusher.outputs

{'pushed_model': Channel(
    type_name: PushedModel
    artifacts: [Artifact(artifact: id: 7
type_id: 16
uri: "/content/tfx/Pusher/pushed_model/6"
custom_properties {
  key: "name"
  value {
    string_value: "pushed_model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Pusher"
  }
}
custom_properties {
  key: "pushed"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "pushed_destination"
  value {
    string_value: "/content/tfx/serving_model/1620861979"
  }
}
custom_properties {
  key: "pushed_version"
  value {
    string_value: "1620861979"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "0.29.0"
  }
}
state: LIVE
, artifact_type: id: 16
name: "PushedModel"
)]
    additional_properties: {}
    additional_custom_properties: {}
)}

In [29]:
push_uri = pusher.outputs.pushed_model.get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

('serving_default',
 <ConcreteFunction signature_wrapper(*, input_1) at 0x7F7F8765D510>)


# Full Pipeline

In [ ]:
!rm -rf data.*
# !rm -rf *trainer.py ##EDIT: Python files have to be retained
!rm -rf *.csv
!sudo rm -r /content/tfx

! cd /content/
! mkdir /content/tfx/
! mkdir /content/tfx/pipelines
! mkdir /content/tfx/metadata
! mkdir /content/tfx/logs
! mkdir /content/tfx/data
! mkdir /content/tfx/serving_model

! mkdir /content/train_data/
! mkdir /content/eval_data/
! mkdir /content/test_data/

# !wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

mkdir: cannot create directory ‘/content/train_data/’: File exists
mkdir: cannot create directory ‘/content/eval_data/’: File exists


In [ ]:
##Define all constant
_tfx_root = os.path.join(os.getcwd(), 'tfx');        # Create location ~/tfx
_pipeline_root = os.path.join(_tfx_root, 'pipelines');      # Join ~/tfx/pipelines/
_metadata_db_root = os.path.join(_tfx_root, 'metadata.db');    # Join ~/tfx/metadata.db
_log_root = os.path.join(_tfx_root, 'logs');
_model_root = os.path.join(_tfx_root, 'model');
_data_root = os.path.join(_tfx_root, 'data');
_serving_model_dir = os.path.join(_tfx_root, 'serving_model')
_data_filepath = os.path.join(_data_root, "data_trans.csv")

_input_fn_module_file = 'inputfn_trainer.py'
_constants_module_file = 'constants_trainer.py'
_model_trainer_module_file = 'model_trainer.py'

context = InteractiveContext(pipeline_root=_tfx_root)

In [ ]:
# my file path, just like the picture above
cwd = "drive/My Drive/CMP-258 Project/dataset/SampledDatasetFrames/"
# the tfrecord file path, you need to create the folder yourself
recordPath = ""
# the best number of images stored in each tfrecord file
bestNum = 1000
# the index of images flowing into each tfrecord file
num = 0
# the index of the tfrecord file
recordFileNum = 0
# the number of classes of images
keys = ['NoFight', 'Fight']
values = [0, 1]
classes = dict(zip(keys, values))
# name format of the tfrecord files
recordFileName = ("tfx/data/train.tfrecords")
# tfrecord file writer
writer = tf.io.TFRecordWriter(recordFileName)

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def read_image_as_byte_string(path):
  # "image/raw" has bytes value "None" which cannot be decoded as a UTF-8 string.
  image_string = open(str(path), 'rb').read()
  return image_string

print("Creating the 000 tfrecord file")
for name, label in classes.items():
    print(name)
    print(label)
    class_path = os.path.join(cwd, name)
    for img_name in os.listdir(class_path):
        num += 1
        if num > bestNum:
            num = 1
            recordFileNum += 1
            writer = tf.io.TFRecordWriter(recordPath + recordFileNum)
            print("Creating the %.3d tfrecord file" % recordFileNum)
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path, "r")
        img_raw = img.tobytes()
        image_string = read_image_as_byte_string(img_path)
        example = tf.train.Example(features=tf.train.Features(feature={
          "img_raw": _bytes_feature(image_string),
          # "img_raw": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
          "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))}))
        writer.write(example.SerializeToString())
writer.close()

Creating the 000 tfrecord file
NoFight
0
Fight
1


In [ ]:
def create_final_pipeline(
    pipeline_name: Text,
    root_path: Text,
    data_path: Text,
    training_params: Dict[Text, Text],
    # beam_pipeline_args: List[Text],
) -> pipeline.Pipeline:

  _pipeline_root = os.path.join(root_path, 'pipelines');      # Join ~/tfx/pipelines/
  _metadata_db_root = os.path.join(root_path, 'metadata.db');    # Join ~/tfx/metadata.db
  _log_root = os.path.join(root_path, 'logs');
  _model_root = os.path.join(root_path, 'model');
  _serving_model_dir = os.path.join(root_path, 'serving_model')

  print("-----Example Gen-----")
  tf_record_gen = ImportExampleGen(input=external_input(data_path))
  print("-----Statistics Gen-----")
  statistics_gen = StatisticsGen(examples=tf_record_gen.outputs['examples'])
  print("-----Schema Gen-----")
  schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],infer_feature_shape=False)
  print("-----Validator Gen-----")
  example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'],schema=schema_gen.outputs['schema'])
  print("-----Trainer-----")
  trainer = Trainer(
    module_file=os.path.abspath(_model_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=tf_record_gen.outputs['examples'],
    train_args=trainer_pb2.TrainArgs(),
    eval_args=trainer_pb2.EvalArgs(),
    custom_config=({"epochs": 1})
    )
  print("-----Pusher-----")
  pusher = Pusher(
    model=trainer.outputs['model'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
  
  return pipeline.Pipeline(
    pipeline_name=  pipeline_name,
    pipeline_root=  root_path,
    components=[
        tf_record_gen, statistics_gen, schema_gen, example_validator,
        trainer, pusher
    ],
    
    metadata_connection_config = metadata.sqlite_metadata_connection_config(_metadata_db_root),
    enable_cache=True,
    beam_pipeline_args=['--direct_num_workers=%d' % 0],
  )

In [ ]:
#Run pipeline locally
from tfx.orchestration.local.local_dag_runner import LocalDagRunner

##Define all paths
_tfx_root = os.path.join(os.getcwd(), 'tfx')

#Config params
training_params = {"epochs": 2}

#Create and run pipeline
p_ = create_final_pipeline(root_path = _tfx_root, 
                           pipeline_name="local_pipeline", 
                           data_path="/content/train_data",
                           training_params=training_params)

LocalDagRunner().run(p_)

-----Example Gen-----


-----Statistics Gen-----
-----Schema Gen-----
-----Validator Gen-----
-----Trainer-----
-----Pusher-----


2.4.1
INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
Inside Trainer
<BatchDataset shapes: ((None, 299, 299, None), (None, 2)), types: (tf.float32, tf.float32)>
Epoch 1/2
14/14 [==============================] - 177s 12s/step - loss: 0.7049 - accuracy: 0.5057 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.7368 - val_accuracy: 0.3383 - val_top_k_categorical_accuracy: 1.0000
Epoch 2/2
14/14 [==============================] - 143s 10s/step - loss: 0.6458 - accuracy: 0.7064 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.7007 - val_accuracy: 0.5489 - val_top_k_categorical_accuracy: 1.0000
INFO:tensorflow:Assets written to: /content/tfx/Trainer/model/3/Format-Serving/assets
